# Imports e Utilidades

In [1]:
import  warnings,                   \
        calendar,                   \
        pandas as pd,               \
        numpy as np,                \
        requests as rt,             \
        hydrobr as hbr,             \
        xml.etree.ElementTree as ET
from typing import List

from datetime import datetime, timedelta

from io import BytesIO

# Desativar as mensagens de 'warning' que ficam poluindo o output de alguns trechos de código.
warnings.filterwarnings("ignore")

In [17]:
def carregar_dados(file_name : str,
                   separator : str = "\t",
                   adjust : bool = True,
                   date_column : str = "ds"
                   ) -> pd.DataFrame:
    
    df = pd.read_csv(file_name, sep=separator, index_col=date_column, header=0, parse_dates=[date_column])

    if adjust:
        df = df.resample('D').first() # deixando a série contínua numa base diária

    # Deixando ajustado para usar com as libs Nixtla
    df['unique_id'] = 1
    df.reset_index(inplace=True)

    return df
# ============================================================================================ #
def get_telemetrica(codEstacao : str,
                    dataInicio : str,
                    dataFim : str,
                    save : bool = False) -> pd.DataFrame:
    # 1. Fazer a requisião ao servidor e pegar a árvore e a raiz dos dados 
    params = {'codEstacao':codEstacao, 'dataInicio':dataInicio, 'dataFim':dataFim}
    server = 'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/DadosHidrometeorologicos'
    response = rt.get(server, params)
    tree = ET.ElementTree(ET.fromstring(response.content))
    root = tree.getroot()

    # 2. Iteração dentro dos elementos do XML procurando os dados que são disponibilizados para a estação
    list_vazao = []
    list_data = []
    list_cota = []
    list_chuva = []

    for i in root.iter('DadosHidrometereologicos'):

        data = i.find('DataHora').text
        try:
            vazao = float(i.find('Vazao').text)
        except TypeError:
            vazao = i.find('Vazao').text

        try:
            cota = float(i.find('Nivel').text)
        except TypeError:
            cota = i.find('Nivel').text

        try:
            chuva = float(i.find('Chuva').text)
        except TypeError:
            chuva = i.find('Chuva').text

        list_vazao.append(vazao)
        list_data.append(data)
        list_cota.append(cota)
        list_chuva.append(chuva)

    df = pd.DataFrame([list_data, list_cota, list_chuva, list_vazao]).transpose()
    df.columns = ['Data', 'Cota', 'Chuva', 'Vazao']
    df = df.sort_values(by='Data')
    df = df.set_index('Data')
    
    if save == True:
        df.to_excel(codEstacao+'_dados_tele.xlsx')
    
    return df
# ============================================================================================ #
def get_convencional(codEstacao : str,
                     dataInicio : str,
                     dataFim : str,
                     tipoDados : int,
                     nivelConsistencia : int,
                     save : bool = False) -> pd.DataFrame:
    """
        Série Histórica estação - HIDRO.
        codEstacao : Código Plu ou Flu
        dataInicio : <YYYY-mm-dd>
        dataFim : Caso não preenchido, trará até o último dado mais recente armazenado
        tipoDados : 1-Cotas, 2-Chuvas ou 3-Vazões
        nivelConsistencia : 1-Bruto ou 2-Consistido
    """

    # 1. Fazer a requisião ao servidor e pegar a árvore e a raiz dos dados 
    params = {'codEstacao':codEstacao, 'dataInicio':dataInicio, 'dataFim':dataFim,
              'tipoDados':tipoDados, 'nivelConsistencia':nivelConsistencia}
    
    server = 'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica'
    response = rt.get(server, params)
    tree = ET.ElementTree(ET.fromstring(response.content))
    root = tree.getroot()
    
    # 2. Iteração dentro dos elementos do XML procurando os dados que são disponibilizados para a estação
    list_data = []
    list_consistenciaF = []
    list_month_dates = []

    for i in root.iter('SerieHistorica'):

        consistencia = i.find('NivelConsistencia').text
        date = i.find('DataHora').text
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        last_day = calendar.monthrange(date.year, date.month)[1]
        month_dates = [date + timedelta(days=i) for i in range(last_day)]
        content = []
        list_consistencia = []

        for day in range(last_day):
            if tipoDados == 1:
                value = f'Cota{day+1:02d}'
            if tipoDados == 2:
                value = f'Chuva{day+1:02d}'
            if tipoDados == 3:
                value = f'Vazao{day+1:02d}'
            
            try:
                content.append(float(i.find(value).text))
                list_consistencia.append(int(consistencia))
            except TypeError:
                content.append(i.find(value).text)
                list_consistencia.append(int(consistencia))
            except AttributeError:
                content.append(None)
                list_consistencia.append(int(consistencia))
        
        list_data += content
        list_consistenciaF += list_consistencia
        list_month_dates += month_dates
    df = pd.DataFrame([list_month_dates, list_consistenciaF, list_data]).transpose()

    if tipoDados == 1:
        df.columns = ['Data','Consistencia','Cota']
    elif tipoDados == 2:
        df.columns = ['Data','Consistencia','Chuva']
    else: # Vazão
        df.columns = ['Data','Consistencia','Vazao']
    
    df = df.sort_values(by='Data')
    df = df.set_index('Data')

    if save == True:
        df.to_excel(codEstacao + '_dados_conv.xlsx')
    
    return df
# ============================================================================================ #
def gerar_dados_tele(estacao_principal : str,
                    outras_estacoes : List[str],
                    nome_arq : str,
                    dt_inicio : str,
                    dt_fim : str,
                    salvar : bool = False) -> None:
    """
            Este método vai pegar o código da 'estacao_principal' (que o usuário já sabe previamente que é uma telemétrica), baixar os dados da estação
        e concatenar (outer join) com os dados das outras estações telemétricas. Neste método já será realizada a conversão dos dados de 'object' para
        os tipos de acordo, ou seja, 'float' para os campos numéricos e 'datetime' para os campos de datahora.
            Como o desejo do trabalho é lidar com dados diários, já aproveita pra fazer a agregação dos dados desta maneira também.
            Após tudo isso, salva num arquivo xlsx para usos posteriores.

        Parâmetros:
            estacao_principal : str,
            outras_estacoes : List[str],
            nome_arq : str,
            dt_inicio : str = 'YYYY-mm-dd',
            dt_fim : str = 'YYYY-mm-dd',
            salvar : bool = True|False
    """

    df_result = get_telemetrica(codEstacao=estacao_principal, dataInicio=dt_inicio, dataFim=dt_fim)

    df_result.index = pd.to_datetime(df_result.index)
    df_result.Cota = pd.to_numeric(df_result.Cota, errors='coerce')
    df_result.Chuva = pd.to_numeric(df_result.Chuva, errors='coerce')
    df_result.Vazao = pd.to_numeric(df_result.Vazao, errors='coerce')

    df_result = df_result.resample('D').agg({'Cota': 'mean', 'Chuva': 'sum', 'Vazao': 'mean'})

    df_result.columns = ['t_ct_'+str(estacao_principal), 't_cv_'+str(estacao_principal), 't_vz_'+str(estacao_principal)]

    # Agora que já tenho os dados da estação que considero principal na análise (target)
    #   vou agregar com os dados das demais estações

    for e in outras_estacoes:
        df_temp = get_telemetrica(codEstacao=e, dataInicio=dt_inicio, dataFim=dt_fim)

        # Convertendo os dados
        df_temp.index = pd.to_datetime(df_temp.index)
        df_temp.Cota = pd.to_numeric(df_temp.Cota, errors='coerce')
        df_temp.Chuva = pd.to_numeric(df_temp.Chuva, errors='coerce')
        df_temp.Vazao = pd.to_numeric(df_temp.Vazao, errors='coerce')

        # Para as telemétricas já agrego aqui mesmo
        df_temp = df_temp.resample('D').agg({'Cota': 'mean', 'Chuva': 'sum', 'Vazao': 'mean'})

        # Ajeito os nomes das colunas pra conter de qual estacao os dado veio
        df_temp.columns = ['t_ct_'+e, 't_cv_'+e, 't_vz_'+e]

        df_result = pd.concat([df_result, df_temp], axis=1)

    if salvar:
        df_result.to_excel(nome_arq+'_dados_tele.xlsx')
# ============================================================================================ #
def gerar_dados_conv(estacao_principal : str,
                    outras_estacoes : List[str],
                    nome_arq : str,
                    dt_inicio : str,
                    dt_fim : str,
                    tp_dados : int,
                    nvl_consistencia : str,
                    drop_consistencia : bool = True, # Remover a coluna "NivelConsistência". Ela será irrelevante, até segunda ordem.
                    salvar : bool = False) -> None:
    """
            Este método vai pegar o código da 'estacao_principal' (que o usuário já sabe previamente que é uma convencional), baixar os dados da estação
        e concatenar (outer join) com os dados das outras estações convencionais. Neste método já será realizada a conversão dos dados de 'object' para
        os tipos de acordo, ou seja, 'float' para os campos numéricos e 'datetime' para os campos de datahora.
            Como o desejo do trabalho é lidar com dados diários, já aproveita pra fazer a agregação dos dados desta maneira também.
            Após tudo isso, salva num arquivo xlsx para usos posteriores.

        Parâmetros:
            estacao_principal : str,
            outras_estacoes : List[str],
            nome_arq : str,
            dt_inicio : str = 'YYYY-mm-dd',
            dt_fim : str = 'YYYY-mm-dd',
            tp_dados : int (1-cota | 2-chuva | 3-vazao),
            nvl_consistencia : int (1-bruto | 2-consistido),
            drop_consistencia : bool = True, (Remover a coluna "NivelConsistência". Ela será irrelevante, até segunda ordem)
            salvar : bool = False
    """

    df_result = get_convencional(codEstacao=estacao_principal, dataInicio=dt_inicio, dataFim=dt_fim, tipoDados=tp_dados, nivelConsistencia=nvl_consistencia)

    df_result.index = pd.to_datetime(df_result.index)

    if drop_consistencia:
        df_result.drop(columns=['Consistencia'], inplace=True)

    if tp_dados == 1:
        df_result.Cota = pd.to_numeric(df_result.Cota, errors='coerce')
        df_result = df_result.resample('D').agg({'Cota': 'mean'})
        df_result.columns = ['c_ct_'+str(estacao_principal)]
    elif tp_dados == 2:
        df_result.Chuva = pd.to_numeric(df_result.Chuva, errors='coerce')
        df_result = df_result.resample('D').agg({'Chuva': 'sum'})
        df_result.columns = ['c_cv_'+str(estacao_principal)]
    else: # Vazão
        df_result.Vazao = pd.to_numeric(df_result.Vazao, errors='coerce')
        df_result = df_result.resample('D').agg({'Vazao': 'mean'})
        df_result.columns = ['c_vz_'+str(estacao_principal)]

    # Agora que já tenho os dados da estação que considero principal na análise (target)
    #   vou agregar com os dados das demais estações

    for e in outras_estacoes:
        df_temp = get_convencional(codEstacao=e, dataInicio=dt_inicio, dataFim=dt_fim, tipoDados=tp_dados, nivelConsistencia=nvl_consistencia)

        # Convertendo os dados
        df_temp.index = pd.to_datetime(df_temp.index)

        if drop_consistencia:
            df_temp.drop(columns=['Consistencia'], inplace=True)

        if tp_dados == 1:
            df_temp.Cota = pd.to_numeric(df_temp.Cota, errors='coerce')
            df_temp = df_temp.resample('D').agg({'Cota': 'mean'})
            df_temp.columns = ['c_ct_'+str(e)]
        elif tp_dados == 2:
            df_temp.Chuva = pd.to_numeric(df_temp.Chuva, errors='coerce')
            df_temp = df_temp.resample('D').agg({'Chuva': 'sum'})
            df_temp.columns = ['c_cv_'+str(e)]
        else: # Vazão
            df_temp.Vazao = pd.to_numeric(df_temp.Vazao, errors='coerce')
            df_temp = df_temp.resample('D').agg({'Vazao': 'mean'})
            df_temp.columns = ['c_vz_'+str(e)]

        df_result = pd.concat([df_result, df_temp], axis=1)

    if salvar:
        if tp_dados == 1:
            df_result.to_excel(nome_arq + '_dados_cota_conv.xlsx')
        elif tp_dados == 2:
            df_result.to_excel(nome_arq + '_dados_chuva_conv.xlsx')
        else:
            df_result.to_excel(nome_arq + '_dados_vazao_conv.xlsx')

# Download dos dados

# Alto Rio Doce

In [5]:
# Baixando os dados das estações que serão utilizadas no trabalho
# As estações foram selecionadas a partir do sistema Data Rhama
# Aqui eu baixo os dados e salvo localmente
# >>>>>>>>>>>>> SÓ PRECISA FAZER ISSO UMA VEZ, POR ISSO O CÓDIGO FICA COMENTADO DEPOIS DE RODAR!!!! <<<<<<<<<<<<<

estacao_principal = '56425000'
outras_estacoes = ['56338500', '56338080', '56110005', '56337200', '56337500']

## Telemétricas

In [6]:
# Aplicando a lib HydroBR eu desejo saber se as estações em questão são do tipo convencional ou telemétrica
# O código não exclui o fato, eventual, de uma dada estação ser convencional E telemétrica, como é o caso aqui

lista_estacoes = hbr.get_data.ANA.list_telemetric() # Vendo primeiro se tem telemétrica
lista_estacoes.head()

4680it [00:07, 646.22it/s] 


,Name,Code,Status,SubBasin,City-State,Origem,Responsible,Elevation,Latitude,Longitude
1,UHE BELO MONTE BR230,00351004,Ativo,18,VITÓRIA DO XINGU-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,33.00,-3.1267,-51.7906
2,UHE BELO MONTE SÍTIO PIMENTAL,00351005,Ativo,18,VITÓRIA DO XINGU-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,110.00,-3.3758,-51.9403
3,UHE BELO MONTE VISTA ALEGRE,00352009,Ativo,18,ALTAMIRA-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,125.00,-3.1186,-52.2525
4,UHE BELO MONTE SÃO FRANCISCO,00352010,Ativo,18,ALTAMIRA-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,124.00,-3.2533,-52.3489
5,UHE BELO MONTE JOARI,00652002,Ativo,18,SÃO FÉLIX DO XINGÚ-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,0.00,-6.5603,-52.0764


In [7]:
# Averiguando se as estações que tenho em mãos estão presentes neste conjunto de estações telemétricas

print("Estação {e} -> {p}".format(
                                e=estacao_principal,
                                p=(lista_estacoes['Code'] == estacao_principal).any()
                            )
)

# A estação principal em questão tem dados telemétricos

Estação 56425000 -> True


In [8]:
# Verificando as outras estações

for e in outras_estacoes:
    print("Estação {e} -> {p}".format(
                                        e=e,
                                        p=(lista_estacoes['Code'] == e).any()
                                    )
    )

# Estas estações também têm dados telemétricos

Estação 56338500 -> True
Estação 56338080 -> True
Estação 56110005 -> True
Estação 56337200 -> True
Estação 56337500 -> True


In [9]:
# Gerando um arquivo Excel com os dados das estações telemétricas

gerar_dados_tele(
    estacao_principal=estacao_principal,
    outras_estacoes=outras_estacoes,
    nome_arq="alto_rio_doce",
    dt_inicio='2013-01-01',
    dt_fim='2023-12-31',
    salvar=True
)

## Convencionais

### Cota/Vazão

In [10]:
# Aplicando a lib HydroBR eu desejo saber se as estações em questão são do tipo convencional ou telemétrica
# O código não exclui o fato, eventual, de uma dada estação ser convencional E telemétrica, como é o caso aqui

lista_estacoes = hbr.get_data.ANA.list_flow( # Verificando se tem estações de cota/vazão primeiro
    state='MINAS GERAIS',
    source='ANA'
)

lista_estacoes.head()

2735it [00:02, 1081.87it/s]


,Name,Code,Type,DrainageArea,SubBasin,City,State,Responsible,Latitude,Longitude
1,SÃO ROQUE DE MINAS,40023000,1,None,40,SÃO ROQUE DE MINAS,MINAS GERAIS,IGAM-MG,-20.3344,-46.4697
2,VARGEM BONITA,40025000,1,299,40,VARGEM BONITA,MINAS GERAIS,ANA,-20.3272,-46.3661
3,IGUATAMA,40027000,1,None,40,IGUATAMA,MINAS GERAIS,IGAM-MG,-20.1717,-45.7261
4,FAZENDA DA BARCA,40030000,1,725,40,SÃO ROQUE DE MINAS,MINAS GERAIS,ANA,-20.1000,-46.3167
5,FAZENDA SAMBURÁ,40032000,1,754,40,SÃO ROQUE DE MINAS,MINAS GERAIS,ANA,-20.1508,-46.3033


In [11]:
# Averiguando se as estações que tenho em mãos estão presentes neste conjunto de estações convencionais de cota/vazão

print("Estação {e} -> {p}".format(
        e=estacao_principal,
        p=(lista_estacoes['Code'] == estacao_principal).any()
    )
)

# A estação principal tem dados convencionais de cota/vazão

Estação 56425000 -> True


In [12]:
# Verificando as outras estações

for e in outras_estacoes:
    print("Estação {e} -> {p}".format(
            e=e,
            p=(lista_estacoes['Code'] == e).any()
        )
    )

# Estas estações também têm dados convencionais de cota/vazão

Estação 56338500 -> True
Estação 56338080 -> True
Estação 56110005 -> True
Estação 56337200 -> True
Estação 56337500 -> True


In [13]:
# Gerando um arquivo Excel com os dados das estações convencionais

gerar_dados_conv(
    estacao_principal=estacao_principal,
    outras_estacoes=outras_estacoes,
    nome_arq="alto_rio_doce",
    dt_inicio='2013-01-01',
    dt_fim='2023-12-31',
    tp_dados=1, # Cota
    nvl_consistencia='2', # dados consistidos
    salvar=True
)

In [14]:
# Gerando um arquivo Excel com os dados das estações convencionais

gerar_dados_conv(
    estacao_principal=estacao_principal,
    outras_estacoes=outras_estacoes,
    nome_arq="alto_rio_doce",
    dt_inicio='2013-01-01',
    dt_fim='2023-12-31',
    tp_dados=3, # Vazão
    nvl_consistencia='2', # dados consistidos
    salvar=True
)

### Chuva

In [15]:
lista_estacoes = hbr.get_data.ANA.list_prec(
    state='MINAS GERAIS',
    source='ANA'
)

lista_estacoes.head()

2413it [00:02, 1189.82it/s]


,Name,Code,Type,SubBasin,City,State,Responsible,Latitude,Longitude
1,AÇUDE DO ESTREITO,01442019,2,44,ESPINOSA,MINAS GERAIS,DNOCS,-14.8167,-42.8000
2,ESPIGÃO,01442020,2,44,ESPINOSA,MINAS GERAIS,SUDENE,-14.9833,-42.5667
3,ITAMIRIM,01442021,2,44,ESPINOSA,MINAS GERAIS,SUDENE,-14.7667,-42.8833
4,ESPINOSA,01442022,2,44,ESPINOSA,MINAS GERAIS,SUDENE,-14.9333,-42.8167
5,ESPINOSA,01442025,2,44,ESPINOSA,MINAS GERAIS,INMET,-14.9333,-42.8167


In [16]:
print("Estação {e} -> {p}".format(
        e=estacao_principal,
        p=(lista_estacoes['Code'] == estacao_principal).any()
    )
)

# A estação principal NÃO tem dados convencionais de chuva

Estação 56425000 -> False


In [17]:
# Verificando as outras estações

for e in outras_estacoes:
    print("Estação {e} -> {p}".format(
            e=e,
            p=(lista_estacoes['Code'] == e).any()
        )
    )

# Estas estações também NÃO têm dados convencionais de chuva

Estação 56338500 -> False
Estação 56338080 -> False
Estação 56110005 -> False
Estação 56337200 -> False
Estação 56337500 -> False


Já tenho todos os dados de que preciso. Estão salvos localmente, agora é carregar em memória, juntar tudo e mandar ver nos experimentos.

## Juntando os dados

In [18]:
# Como estou em dúvida com o arquivo de cotas, deixei apenas estes dois arquivos pra trabalhar.

arquivos = ['alto_rio_doce_dados_tele.xlsx', 'alto_rio_doce_dados_vazao_conv.xlsx']

In [19]:
# Vou fazer a carga primeiro dos dados telemétricos, porque é onde tem mais informação de uma única vez.
# Depois concateno os outros arquivos. Mas a ordem tanto faz aqui, só estipulei assim porque acho melhor

df = pd.read_excel(arquivos[0], sheet_name=0, index_col=0, header=0, parse_dates=['Data'])

for a in range(1, len(arquivos)):
    df_temp = pd.read_excel(arquivos[a], sheet_name=0, index_col=0, header=0, parse_dates=['Data'])
    df = pd.concat([df, df_temp], axis=1)

df

,t_ct_56425000,t_cv_56425000,t_vz_56425000,t_ct_56338500,t_cv_56338500,t_vz_56338500,t_ct_56338080,t_cv_56338080,t_vz_56338080,t_ct_56110005,...,t_vz_56337200,t_ct_56337500,t_cv_56337500,t_vz_56337500,c_vz_56425000,c_vz_56338500,c_vz_56338080,c_vz_56110005,c_vz_56337200,c_vz_56337500
Data,,,,,,,,,,,,,,,,,,,,,
2013-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,82.7871,NaN,NaN,60.0561,NaN,NaN
2013-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.642708,...,NaN,NaN,NaN,NaN,80.4893,NaN,NaN,37.5463,NaN,NaN
2013-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.339583,...,NaN,NaN,NaN,NaN,78.2142,NaN,NaN,39.2282,NaN,NaN
2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.964583,...,NaN,NaN,NaN,NaN,79.3489,NaN,NaN,37.5463,NaN,NaN
2013-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.552083,...,NaN,NaN,NaN,NaN,129.1610,NaN,NaN,39.2282,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,140.072917,8.2,88.387500,98.583333,0.2,99.441667,32769.083333,0.0,NaN,127.967391,...,0.600000,80.458333,8.6,28.054167,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-28,139.354167,2.0,87.527083,93.041667,1.2,93.541667,33148.863636,0.0,NaN,117.815217,...,33.758333,68.541667,0.8,21.679167,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-29,129.583333,0.0,76.317708,77.695652,0.0,77.721739,33278.208333,0.0,NaN,107.022727,...,61.954167,67.166667,0.0,21.008333,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.columns

Index(['t_ct_56425000', 't_cv_56425000', 't_vz_56425000', 't_ct_56338500',
       't_cv_56338500', 't_vz_56338500', 't_ct_56338080', 't_cv_56338080',
       't_vz_56338080', 't_ct_56110005', 't_cv_56110005', 't_vz_56110005',
       't_ct_56337200', 't_cv_56337200', 't_vz_56337200', 't_ct_56337500',
       't_cv_56337500', 't_vz_56337500', 'c_vz_56425000', 'c_vz_56338500',
       'c_vz_56338080', 'c_vz_56110005', 'c_vz_56337200', 'c_vz_56337500'],
      dtype='object')

In [21]:
# Vou remover as colunas das cotas

df.drop(columns=['t_ct_56425000', 't_ct_56338500', 't_ct_56338080', 't_ct_56110005', 't_ct_56337200', 't_ct_56337500'], inplace=True)
df

,t_cv_56425000,t_vz_56425000,t_cv_56338500,t_vz_56338500,t_cv_56338080,t_vz_56338080,t_cv_56110005,t_vz_56110005,t_cv_56337200,t_vz_56337200,t_cv_56337500,t_vz_56337500,c_vz_56425000,c_vz_56338500,c_vz_56338080,c_vz_56110005,c_vz_56337200,c_vz_56337500
Data,,,,,,,,,,,,,,,,,,
2013-01-01,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,82.7871,NaN,NaN,60.0561,NaN,NaN
2013-01-02,NaN,NaN,NaN,NaN,NaN,NaN,0.0,46.950000,NaN,NaN,NaN,NaN,80.4893,NaN,NaN,37.5463,NaN,NaN
2013-01-03,NaN,NaN,NaN,NaN,NaN,NaN,0.0,46.703125,NaN,NaN,NaN,NaN,78.2142,NaN,NaN,39.2282,NaN,NaN
2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,0.0,49.239583,NaN,NaN,NaN,NaN,79.3489,NaN,NaN,37.5463,NaN,NaN
2013-01-05,NaN,NaN,NaN,NaN,NaN,NaN,0.0,49.780208,NaN,NaN,NaN,NaN,129.1610,NaN,NaN,39.2282,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,8.2,88.387500,0.2,99.441667,0.0,NaN,0.0,70.576087,2.2,0.600000,8.6,28.054167,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-28,2.0,87.527083,1.2,93.541667,0.0,NaN,0.0,60.023913,0.0,33.758333,0.8,21.679167,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-29,0.0,76.317708,0.0,77.721739,0.0,NaN,0.0,49.910227,0.0,61.954167,0.0,21.008333,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df.columns

Index(['t_cv_56425000', 't_vz_56425000', 't_cv_56338500', 't_vz_56338500',
       't_cv_56338080', 't_vz_56338080', 't_cv_56110005', 't_vz_56110005',
       't_cv_56337200', 't_vz_56337200', 't_cv_56337500', 't_vz_56337500',
       'c_vz_56425000', 'c_vz_56338500', 'c_vz_56338080', 'c_vz_56110005',
       'c_vz_56337200', 'c_vz_56337500'],
      dtype='object')

In [23]:
# Fazendo o merge das colunas de vazão que são correspondentes à mesma estação
# Acontece que existem gaps entre os dados, o que é estranho, porque a estação telemétrica tem dados que a convencional não tem e vice-versa.
# Vou contar qual coluna tem mais dados e depois executar um 'fillna'
colunas_esquerda = ['t_vz_56425000', 't_vz_56338500', 't_vz_56338080', 't_vz_56110005', 't_vz_56337200', 't_vz_56337500']
colunas_direita = ['c_vz_56425000', 'c_vz_56338500', 'c_vz_56338080', 'c_vz_56110005', 'c_vz_56337200', 'c_vz_56337500']

for i, j in zip(colunas_esquerda, colunas_direita):
    print(i, j, df[i].isna().sum(), df[j].isna().sum())

# A coluna que tiver menos buracos será preenchida com os dados da coluna que tem mais dados faltantes

t_vz_56425000 c_vz_56425000 1704 92
t_vz_56338500 c_vz_56338500 1432 4017
t_vz_56338080 c_vz_56338080 4004 4017
t_vz_56110005 c_vz_56110005 197 245
t_vz_56337200 c_vz_56337200 1439 4017
t_vz_56337500 c_vz_56337500 1932 4017


In [24]:
# Preenchendo a coluna que tem menos dados faltantes com a outra correspondente
# Fazer isso para cada coluna, contudo, tem colunas que tem dados faltando demais. Neste caso, darei drop nelas inteiramente.
df['c_vz_56425000'].fillna(df['t_vz_56425000'], inplace=True)
df['t_vz_56110005'].fillna(df['c_vz_56110005'], inplace=True)

df['c_vz_56425000'].isna().sum(), df['t_vz_56110005'].isna().sum()

(0, 0)

In [25]:
# Uma vez que as colunas estejam ajustadas, eu dropo as que não vou precisar mais
df = df.drop(columns=['t_vz_56425000', 'c_vz_56110005'])
df

,t_cv_56425000,t_cv_56338500,t_vz_56338500,t_cv_56338080,t_vz_56338080,t_cv_56110005,t_vz_56110005,t_cv_56337200,t_vz_56337200,t_cv_56337500,t_vz_56337500,c_vz_56425000,c_vz_56338500,c_vz_56338080,c_vz_56337200,c_vz_56337500
Data,,,,,,,,,,,,,,,,
2013-01-01,NaN,NaN,NaN,NaN,NaN,0.0,60.056100,NaN,NaN,NaN,NaN,82.787100,NaN,NaN,NaN,NaN
2013-01-02,NaN,NaN,NaN,NaN,NaN,0.0,46.950000,NaN,NaN,NaN,NaN,80.489300,NaN,NaN,NaN,NaN
2013-01-03,NaN,NaN,NaN,NaN,NaN,0.0,46.703125,NaN,NaN,NaN,NaN,78.214200,NaN,NaN,NaN,NaN
2013-01-04,NaN,NaN,NaN,NaN,NaN,0.0,49.239583,NaN,NaN,NaN,NaN,79.348900,NaN,NaN,NaN,NaN
2013-01-05,NaN,NaN,NaN,NaN,NaN,0.0,49.780208,NaN,NaN,NaN,NaN,129.161000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,8.2,0.2,99.441667,0.0,NaN,0.0,70.576087,2.2,0.600000,8.6,28.054167,88.387500,NaN,NaN,NaN,NaN
2023-12-28,2.0,1.2,93.541667,0.0,NaN,0.0,60.023913,0.0,33.758333,0.8,21.679167,87.527083,NaN,NaN,NaN,NaN
2023-12-29,0.0,0.0,77.721739,0.0,NaN,0.0,49.910227,0.0,61.954167,0.0,21.008333,76.317708,NaN,NaN,NaN,NaN


In [26]:
# As colunas que, nesta análise, resolvi remover também porque não farão diferença pro trabalho
df = df.drop(columns=['c_vz_56338500', 't_vz_56338080', 'c_vz_56338080', 'c_vz_56337200', 'c_vz_56337500'])
df

,t_cv_56425000,t_cv_56338500,t_vz_56338500,t_cv_56338080,t_cv_56110005,t_vz_56110005,t_cv_56337200,t_vz_56337200,t_cv_56337500,t_vz_56337500,c_vz_56425000
Data,,,,,,,,,,,
2013-01-01,NaN,NaN,NaN,NaN,0.0,60.056100,NaN,NaN,NaN,NaN,82.787100
2013-01-02,NaN,NaN,NaN,NaN,0.0,46.950000,NaN,NaN,NaN,NaN,80.489300
2013-01-03,NaN,NaN,NaN,NaN,0.0,46.703125,NaN,NaN,NaN,NaN,78.214200
2013-01-04,NaN,NaN,NaN,NaN,0.0,49.239583,NaN,NaN,NaN,NaN,79.348900
2013-01-05,NaN,NaN,NaN,NaN,0.0,49.780208,NaN,NaN,NaN,NaN,129.161000
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,8.2,0.2,99.441667,0.0,0.0,70.576087,2.2,0.600000,8.6,28.054167,88.387500
2023-12-28,2.0,1.2,93.541667,0.0,0.0,60.023913,0.0,33.758333,0.8,21.679167,87.527083
2023-12-29,0.0,0.0,77.721739,0.0,0.0,49.910227,0.0,61.954167,0.0,21.008333,76.317708


In [27]:
# Deixando os dados contínuos, numa base diária.
df = df.resample('D').first()
df

,t_cv_56425000,t_cv_56338500,t_vz_56338500,t_cv_56338080,t_cv_56110005,t_vz_56110005,t_cv_56337200,t_vz_56337200,t_cv_56337500,t_vz_56337500,c_vz_56425000
Data,,,,,,,,,,,
2013-01-01,NaN,NaN,NaN,NaN,0.0,60.056100,NaN,NaN,NaN,NaN,82.787100
2013-01-02,NaN,NaN,NaN,NaN,0.0,46.950000,NaN,NaN,NaN,NaN,80.489300
2013-01-03,NaN,NaN,NaN,NaN,0.0,46.703125,NaN,NaN,NaN,NaN,78.214200
2013-01-04,NaN,NaN,NaN,NaN,0.0,49.239583,NaN,NaN,NaN,NaN,79.348900
2013-01-05,NaN,NaN,NaN,NaN,0.0,49.780208,NaN,NaN,NaN,NaN,129.161000
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,8.2,0.2,99.441667,0.0,0.0,70.576087,2.2,0.600000,8.6,28.054167,88.387500
2023-12-28,2.0,1.2,93.541667,0.0,0.0,60.023913,0.0,33.758333,0.8,21.679167,87.527083
2023-12-29,0.0,0.0,77.721739,0.0,0.0,49.910227,0.0,61.954167,0.0,21.008333,76.317708


## Exportando os dados finais

In [28]:
# Neste momento, tenho o DataFrame com os dados EXATAMENTE da forma que preciso.
# Posso, inclusive, exportar isso para um arquivo de Excel
# É o que farei, pois se precisar retornar aos dados originais, será mais fácil que fazer toda engenharia até aqui

df.to_excel('./arquivos_finais/alto_rio_doce_final.xlsx')

# Médio Rio Doce

In [3]:
# Baixando os dados das estações que serão utilizadas no trabalho
# As estações foram selecionadas a partir do sistema Data Rhama
# Aqui eu baixo os dados e salvo localmente
# >>>>>>>>>>>>> SÓ PRECISA FAZER ISSO UMA VEZ, POR ISSO O CÓDIGO FICA COMENTADO DEPOIS DE RODAR!!!! <<<<<<<<<<<<<

estacao_principal = '56920000'
outras_estacoes = ['01941018', '56846900', '56846890', '01841011', '56850000', '56846200', '56895000', '01841020', '01841029']

## Telemétricas

In [4]:
# Aplicando a lib HydroBR eu desejo saber se as estações em questão são do tipo convencional ou telemétrica
# O código não exclui o fato, eventual, de uma dada estação ser convencional E telemétrica, como é o caso aqui

lista_estacoes = hbr.get_data.ANA.list_telemetric() # Vendo primeiro se tem telemétrica
lista_estacoes.head()

4733it [00:02, 1814.08it/s]


,Name,Code,Status,SubBasin,City-State,Origem,Responsible,Elevation,Latitude,Longitude
1,RIO PRETO DA EVA,00259004,Ativo,15,RIO PRETO DA EVA-AM,Açudes Semiárido,00001 - ANA - Agência Nacional de Águas,0.00,-2.7003,-59.6997
2,UHE BELO MONTE BR230,00351004,Ativo,18,VITÓRIA DO XINGU-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,33.00,-3.1267,-51.7906
3,UHE BELO MONTE SÍTIO PIMENTAL,00351005,Ativo,18,VITÓRIA DO XINGU-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,110.00,-3.3758,-51.9403
4,UHE BELO MONTE VISTA ALEGRE,00352009,Ativo,18,ALTAMIRA-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,125.00,-3.1186,-52.2525
5,UHE BELO MONTE SÃO FRANCISCO,00352010,Ativo,18,ALTAMIRA-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,124.00,-3.2533,-52.3489


In [5]:
# Averiguando se as estações que tenho em mãos estão presentes neste conjunto de estações telemétricas

print(
    "Estação {e} -> {p}".format(
        e=estacao_principal,
        p=(lista_estacoes['Code'] == estacao_principal).any()
    )
)

# A estação principal em questão tem dados telemétricos

Estação 56920000 -> True


In [6]:
# Verificando as outras estações

for e in outras_estacoes:
    print(
        "Estação {e} -> {p}".format(
            e=e,
            p=(lista_estacoes['Code'] == e).any()
        )
    )

# Estas estações também têm dados telemétricos

Estação 01941018 -> False
Estação 56846900 -> False
Estação 56846890 -> False
Estação 01841011 -> False
Estação 56850000 -> True
Estação 56846200 -> True
Estação 56895000 -> True
Estação 01841020 -> False
Estação 01841029 -> True


In [7]:
# Gerando um arquivo Excel com os dados das estações telemétricas

estacoes_tele = ["56850000", "56846200", "56895000", "01841029"]
gerar_dados_tele(
    estacao_principal=estacao_principal,
    outras_estacoes=estacoes_tele,
    nome_arq="medio_rio_doce",
    dt_inicio='2013-01-01',
    dt_fim='2023-12-31',
    salvar=True
)

## Convencionais

### Cota/Vazão

In [8]:
# Aplicando a lib HydroBR eu desejo saber se as estações em questão são do tipo convencional ou telemétrica
# O código não exclui o fato, eventual, de uma dada estação ser convencional E telemétrica, como é o caso aqui

lista_estacoes = hbr.get_data.ANA.list_flow( # Verificando se tem estações de cota/vazão primeiro
    state='MINAS GERAIS',
    source='ANA'
)

lista_estacoes.head()

2744it [00:01, 2035.89it/s]


,Name,Code,Type,DrainageArea,SubBasin,City,State,Responsible,Latitude,Longitude
1,SÃO ROQUE DE MINAS,40023000,1,None,40,SÃO ROQUE DE MINAS,MINAS GERAIS,IGAM-MG,-20.3344,-46.4697
2,VARGEM BONITA,40025000,1,299,40,VARGEM BONITA,MINAS GERAIS,ANA,-20.3272,-46.3661
3,IGUATAMA,40027000,1,None,40,IGUATAMA,MINAS GERAIS,IGAM-MG,-20.1717,-45.7261
4,FAZENDA DA BARCA,40030000,1,725,40,SÃO ROQUE DE MINAS,MINAS GERAIS,ANA,-20.1000,-46.3167
5,FAZENDA SAMBURÁ,40032000,1,754,40,SÃO ROQUE DE MINAS,MINAS GERAIS,ANA,-20.1508,-46.3033


In [9]:
# Averiguando se as estações que tenho em mãos estão presentes neste conjunto de estações convencionais de cota/vazão

print(
    "Estação {e} -> {p}".format(
        e=estacao_principal,
        p=(lista_estacoes['Code'] == estacao_principal).any()
    )
)

# A estação principal tem dados convencionais de cota/vazão

Estação 56920000 -> True


In [10]:
# Verificando as outras estações

for e in outras_estacoes:
    print(
        "Estação {e} -> {p}".format(
            e=e,
            p=(lista_estacoes['Code'] == e).any()
        )
    )

# Estas estações também têm dados convencionais de cota/vazão

Estação 01941018 -> False
Estação 56846900 -> True
Estação 56846890 -> True
Estação 01841011 -> False
Estação 56850000 -> True
Estação 56846200 -> True
Estação 56895000 -> True
Estação 01841020 -> False
Estação 01841029 -> False


In [11]:
# Gerando um arquivo Excel com os dados das estações convencionais

estacoes_conv = ["56846900", "56846890", "56850000", "56846200", "56895000"]
gerar_dados_conv(
    estacao_principal=estacao_principal,
    outras_estacoes=estacoes_conv,
    nome_arq="medio_rio_doce",
    dt_inicio='2013-01-01',
    dt_fim='2023-12-31',
    tp_dados=1, # Cota
    nvl_consistencia='2', # dados consistidos
    salvar=True
)

In [12]:
# Gerando um arquivo Excel com os dados das estações convencionais

gerar_dados_conv(
    estacao_principal=estacao_principal,
    outras_estacoes=estacoes_conv,
    nome_arq="medio_rio_doce",
    dt_inicio='2013-01-01',
    dt_fim='2023-12-31',
    tp_dados=3, # Vazão
    nvl_consistencia='2', # dados consistidos
    salvar=True
)

### Chuva

In [13]:
lista_estacoes = hbr.get_data.ANA.list_prec(
    state='MINAS GERAIS',
    source='ANA'
)

lista_estacoes.head()

2422it [00:01, 2233.24it/s]


,Name,Code,Type,SubBasin,City,State,Responsible,Latitude,Longitude
1,AÇUDE DO ESTREITO,01442019,2,44,ESPINOSA,MINAS GERAIS,DNOCS,-14.8167,-42.8000
2,ESPIGÃO,01442020,2,44,ESPINOSA,MINAS GERAIS,SUDENE,-14.9833,-42.5667
3,ITAMIRIM,01442021,2,44,ESPINOSA,MINAS GERAIS,SUDENE,-14.7667,-42.8833
4,ESPINOSA,01442022,2,44,ESPINOSA,MINAS GERAIS,SUDENE,-14.9333,-42.8167
5,ESPINOSA,01442025,2,44,ESPINOSA,MINAS GERAIS,INMET,-14.9333,-42.8167


In [14]:
print(
    "Estação {e} -> {p}".format(
        e=estacao_principal,
        p=(lista_estacoes['Code'] == estacao_principal).any()
    )
)

# A estação principal NÃO tem dados convencionais de chuva

Estação 56920000 -> False


In [15]:
# Verificando as outras estações

for e in outras_estacoes:
    print(
        "Estação {e} -> {p}".format(
            e=e,
            p=(lista_estacoes['Code'] == e).any()
        )
    )

# Estas estações também NÃO têm dados convencionais de chuva

Estação 01941018 -> True
Estação 56846900 -> False
Estação 56846890 -> False
Estação 01841011 -> True
Estação 56850000 -> False
Estação 56846200 -> False
Estação 56895000 -> False
Estação 01841020 -> True
Estação 01841029 -> True


In [16]:
# Gerando um arquivo Excel com os dados das estações convencionais

estacoes_conv = ["01941018", "01841011", "01841020", "01841029"]
gerar_dados_conv(
    estacao_principal=estacao_principal,
    outras_estacoes=estacoes_conv,
    nome_arq="medio_rio_doce",
    dt_inicio='2013-01-01',
    dt_fim='2023-12-31',
    tp_dados=2, # Chuva
    nvl_consistencia='2', # dados consistidos
    salvar=True
)

## Juntando os dados

In [18]:
# Como estou em dúvida com o arquivo de cotas, deixei apenas estes dois arquivos pra trabalhar.

arquivos = ['medio_rio_doce_dados_tele.xlsx', 'medio_rio_doce_dados_vazao_conv.xlsx', 'medio_rio_doce_dados_chuva_conv.xlsx']

In [19]:
# Vou fazer a carga primeiro dos dados telemétricos, porque é onde tem mais informação de uma única vez.
# Depois concateno os outros arquivos. Mas a ordem tanto faz aqui, só estipulei assim porque acho melhor

df = pd.read_excel(arquivos[0], sheet_name=0, index_col=0, header=0, parse_dates=['Data'])

for a in range(1, len(arquivos)):
    df_temp = pd.read_excel(arquivos[a], sheet_name=0, index_col=0, header=0, parse_dates=['Data'])
    df = pd.concat([df, df_temp], axis=1)

df

,t_ct_56920000,t_cv_56920000,t_vz_56920000,t_ct_56850000,t_cv_56850000,t_vz_56850000,t_ct_56846200,t_cv_56846200,t_vz_56846200,t_ct_56895000,...,t_vz_01841029,c_vz_56920000,c_vz_56846900,c_vz_56846890,c_vz_56850000,c_vz_56846200,c_vz_56895000,c_cv_01941018,c_cv_01841011,c_cv_01841020
Data,,,,,,,,,,,,,,,,,,,,,
2013-01-01,NaN,NaN,NaN,156.182609,0.0,342.352174,NaN,NaN,NaN,NaN,...,NaN,293.488,NaN,NaN,287.043,383.563,NaN,0.0,0.0,0.0
2013-01-02,NaN,NaN,NaN,156.621739,0.0,344.686957,NaN,NaN,NaN,NaN,...,NaN,314.322,NaN,NaN,296.658,360.702,NaN,0.0,0.0,0.0
2013-01-03,NaN,NaN,NaN,152.291667,0.0,323.304167,NaN,NaN,NaN,NaN,...,NaN,286.663,NaN,NaN,263.492,324.340,NaN,0.0,0.0,0.0
2013-01-04,NaN,NaN,NaN,151.521739,0.0,318.830435,NaN,NaN,NaN,NaN,...,NaN,276.538,NaN,NaN,263.492,342.814,NaN,0.0,0.0,0.0
2013-01-05,NaN,NaN,NaN,148.952174,0.0,306.108696,NaN,NaN,NaN,NaN,...,NaN,283.273,NaN,NaN,254.270,304.744,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,107.947917,0.4,314.664583,146.934783,2.2,307.548913,409.541667,7.50,239.316667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2023-12-28,115.452381,0.2,341.591667,164.468750,1.4,396.462500,439.708333,1.75,362.766667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,NaN
2023-12-29,144.304348,0.0,454.973913,177.000000,0.0,463.496875,444.625000,0.00,382.045833,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [20]:
df.columns

Index(['t_ct_56920000', 't_cv_56920000', 't_vz_56920000', 't_ct_56850000',
       't_cv_56850000', 't_vz_56850000', 't_ct_56846200', 't_cv_56846200',
       't_vz_56846200', 't_ct_56895000', 't_cv_56895000', 't_vz_56895000',
       't_ct_01841029', 't_cv_01841029', 't_vz_01841029', 'c_vz_56920000',
       'c_vz_56846900', 'c_vz_56846890', 'c_vz_56850000', 'c_vz_56846200',
       'c_vz_56895000', 'c_cv_01941018', 'c_cv_01841011', 'c_cv_01841020'],
      dtype='object')

In [21]:
# Vou remover as colunas das cotas

df = df.drop(columns=['t_ct_56920000', 't_ct_56850000', 't_ct_56846200', 't_ct_56895000', 't_ct_01841029'])
df

,t_cv_56920000,t_vz_56920000,t_cv_56850000,t_vz_56850000,t_cv_56846200,t_vz_56846200,t_cv_56895000,t_vz_56895000,t_cv_01841029,t_vz_01841029,c_vz_56920000,c_vz_56846900,c_vz_56846890,c_vz_56850000,c_vz_56846200,c_vz_56895000,c_cv_01941018,c_cv_01841011,c_cv_01841020
Data,,,,,,,,,,,,,,,,,,,
2013-01-01,NaN,NaN,0.0,342.352174,NaN,NaN,NaN,NaN,NaN,NaN,293.488,NaN,NaN,287.043,383.563,NaN,0.0,0.0,0.0
2013-01-02,NaN,NaN,0.0,344.686957,NaN,NaN,NaN,NaN,NaN,NaN,314.322,NaN,NaN,296.658,360.702,NaN,0.0,0.0,0.0
2013-01-03,NaN,NaN,0.0,323.304167,NaN,NaN,NaN,NaN,NaN,NaN,286.663,NaN,NaN,263.492,324.340,NaN,0.0,0.0,0.0
2013-01-04,NaN,NaN,0.0,318.830435,NaN,NaN,NaN,NaN,NaN,NaN,276.538,NaN,NaN,263.492,342.814,NaN,0.0,0.0,0.0
2013-01-05,NaN,NaN,0.0,306.108696,NaN,NaN,NaN,NaN,NaN,NaN,283.273,NaN,NaN,254.270,304.744,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,0.4,314.664583,2.2,307.548913,7.50,239.316667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2023-12-28,0.2,341.591667,1.4,396.462500,1.75,362.766667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,NaN
2023-12-29,0.0,454.973913,0.0,463.496875,0.00,382.045833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [22]:
df.columns

Index(['t_cv_56920000', 't_vz_56920000', 't_cv_56850000', 't_vz_56850000',
       't_cv_56846200', 't_vz_56846200', 't_cv_56895000', 't_vz_56895000',
       't_cv_01841029', 't_vz_01841029', 'c_vz_56920000', 'c_vz_56846900',
       'c_vz_56846890', 'c_vz_56850000', 'c_vz_56846200', 'c_vz_56895000',
       'c_cv_01941018', 'c_cv_01841011', 'c_cv_01841020'],
      dtype='object')

In [23]:
# Fazendo o merge das colunas de vazão que são correspondentes à mesma estação
# Acontece que existem gaps entre os dados, o que é estranho, porque a estação telemétrica tem dados que a convencional não tem e vice-versa.
# Vou contar qual coluna tem mais dados e depois executar um 'fillna'

colunas_esquerda = ['t_vz_56920000', 't_vz_56850000', 't_vz_56846200', 't_vz_56895000']
colunas_direita = ['c_vz_56920000', 'c_vz_56850000', 'c_vz_56846200', 'c_vz_56895000']

for i, j in zip(colunas_esquerda, colunas_direita):
    print(i, j, df[i].isna().sum(), df[j].isna().sum())

# A coluna que tiver menos buracos será preenchida com os dados da coluna que tem mais dados faltantes

t_vz_56920000 c_vz_56920000 3908 646
t_vz_56850000 c_vz_56850000 368 133
t_vz_56846200 c_vz_56846200 789 1105
t_vz_56895000 c_vz_56895000 2007 4017


In [24]:
# Preenchendo a coluna que tem menos dados faltantes com a outra correspondente
# Fazer isso para cada coluna, contudo, tem colunas que tem dados faltando demais. Neste caso, darei drop nelas inteiramente.
df['c_vz_56920000'].fillna(df['t_vz_56920000'], inplace=True)
df['c_vz_56850000'].fillna(df['t_vz_56850000'], inplace=True)
df['t_vz_56846200'].fillna(df['c_vz_56846200'], inplace=True)
df['t_vz_56895000'].fillna(df['c_vz_56895000'], inplace=True)

df['c_vz_56920000'].isna().sum(), df['c_vz_56850000'].isna().sum(), df['t_vz_56846200'].isna().sum(), df['t_vz_56895000'].isna().sum()

(621, 0, 6, 2007)

In [25]:
# Uma vez que as colunas estejam ajustadas, eu dropo as que não vou precisar mais
df = df.drop(columns=['t_vz_56920000', 't_vz_56850000', 'c_vz_56846200', 'c_vz_56895000', 't_vz_56895000'])
df

,t_cv_56920000,t_cv_56850000,t_cv_56846200,t_vz_56846200,t_cv_56895000,t_cv_01841029,t_vz_01841029,c_vz_56920000,c_vz_56846900,c_vz_56846890,c_vz_56850000,c_cv_01941018,c_cv_01841011,c_cv_01841020
Data,,,,,,,,,,,,,,
2013-01-01,NaN,0.0,NaN,383.563000,NaN,NaN,NaN,293.488000,NaN,NaN,287.043000,0.0,0.0,0.0
2013-01-02,NaN,0.0,NaN,360.702000,NaN,NaN,NaN,314.322000,NaN,NaN,296.658000,0.0,0.0,0.0
2013-01-03,NaN,0.0,NaN,324.340000,NaN,NaN,NaN,286.663000,NaN,NaN,263.492000,0.0,0.0,0.0
2013-01-04,NaN,0.0,NaN,342.814000,NaN,NaN,NaN,276.538000,NaN,NaN,263.492000,0.0,0.0,0.0
2013-01-05,NaN,0.0,NaN,304.744000,NaN,NaN,NaN,283.273000,NaN,NaN,254.270000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,0.4,2.2,7.50,239.316667,NaN,NaN,NaN,314.664583,NaN,NaN,307.548913,NaN,0.0,NaN
2023-12-28,0.2,1.4,1.75,362.766667,NaN,NaN,NaN,341.591667,NaN,NaN,396.462500,NaN,0.8,NaN
2023-12-29,0.0,0.0,0.00,382.045833,NaN,NaN,NaN,454.973913,NaN,NaN,463.496875,NaN,0.0,NaN


In [26]:
# Verificando a quantidade de valores "NaN" em cada coluna

print("Quantidade de NaN por coluna")
print(df.isna().sum())

print("Percentual de NaN por coluna")
for c in np.asarray(df.columns):
    print(c, (df[c].isna().sum()/len(df))*100)

Quantidade de NaN por coluna
t_cv_56920000    1042
t_cv_56850000       0
t_cv_56846200     781
t_vz_56846200       6
t_cv_56895000    1403
t_cv_01841029    4017
t_vz_01841029    4017
c_vz_56920000     621
c_vz_56846900    3956
c_vz_56846890    1291
c_vz_56850000       0
c_cv_01941018      31
c_cv_01841011       0
c_cv_01841020     122
dtype: int64
Percentual de NaN por coluna
t_cv_56920000 25.939756036843413
t_cv_56850000 0.0
t_cv_56846200 19.442369927806823
t_vz_56846200 0.14936519790888725
t_cv_56895000 34.92656211102813
t_cv_01841029 100.0
t_vz_01841029 100.0
c_vz_56920000 15.459297983569828
c_vz_56846900 98.48145382125965
c_vz_56846890 32.13841175006223
c_vz_56850000 0.0
c_cv_01941018 0.7717201891959173
c_cv_01841011 0.0
c_cv_01841020 3.037092357480707


In [27]:
# Removendo as colunas porque elas têm MUITOS  dados faltantes.

df = df.drop(columns=['t_cv_01841029', 't_vz_01841029', 'c_vz_56846900'])
df

,t_cv_56920000,t_cv_56850000,t_cv_56846200,t_vz_56846200,t_cv_56895000,c_vz_56920000,c_vz_56846890,c_vz_56850000,c_cv_01941018,c_cv_01841011,c_cv_01841020
Data,,,,,,,,,,,
2013-01-01,NaN,0.0,NaN,383.563000,NaN,293.488000,NaN,287.043000,0.0,0.0,0.0
2013-01-02,NaN,0.0,NaN,360.702000,NaN,314.322000,NaN,296.658000,0.0,0.0,0.0
2013-01-03,NaN,0.0,NaN,324.340000,NaN,286.663000,NaN,263.492000,0.0,0.0,0.0
2013-01-04,NaN,0.0,NaN,342.814000,NaN,276.538000,NaN,263.492000,0.0,0.0,0.0
2013-01-05,NaN,0.0,NaN,304.744000,NaN,283.273000,NaN,254.270000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,0.4,2.2,7.50,239.316667,NaN,314.664583,NaN,307.548913,NaN,0.0,NaN
2023-12-28,0.2,1.4,1.75,362.766667,NaN,341.591667,NaN,396.462500,NaN,0.8,NaN
2023-12-29,0.0,0.0,0.00,382.045833,NaN,454.973913,NaN,463.496875,NaN,0.0,NaN


In [28]:
# Deixando os dados contínuos, numa base diária.
df = df.resample('D').first()
df

,t_cv_56920000,t_cv_56850000,t_cv_56846200,t_vz_56846200,t_cv_56895000,c_vz_56920000,c_vz_56846890,c_vz_56850000,c_cv_01941018,c_cv_01841011,c_cv_01841020
Data,,,,,,,,,,,
2013-01-01,NaN,0.0,NaN,383.563000,NaN,293.488000,NaN,287.043000,0.0,0.0,0.0
2013-01-02,NaN,0.0,NaN,360.702000,NaN,314.322000,NaN,296.658000,0.0,0.0,0.0
2013-01-03,NaN,0.0,NaN,324.340000,NaN,286.663000,NaN,263.492000,0.0,0.0,0.0
2013-01-04,NaN,0.0,NaN,342.814000,NaN,276.538000,NaN,263.492000,0.0,0.0,0.0
2013-01-05,NaN,0.0,NaN,304.744000,NaN,283.273000,NaN,254.270000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,0.4,2.2,7.50,239.316667,NaN,314.664583,NaN,307.548913,NaN,0.0,NaN
2023-12-28,0.2,1.4,1.75,362.766667,NaN,341.591667,NaN,396.462500,NaN,0.8,NaN
2023-12-29,0.0,0.0,0.00,382.045833,NaN,454.973913,NaN,463.496875,NaN,0.0,NaN


## Exportando os dados finais

In [29]:
# Neste momento, tenho o DataFrame com os dados EXATAMENTE da forma que preciso.
# Posso, inclusive, exportar isso para um arquivo de Excel
# É o que farei, pois se precisar retornar aos dados originais, será mais fácil que fazer toda engenharia até aqui

df.to_excel('./arquivos_finais/medio_rio_doce_final.xlsx')

# Baixo Rio Doce

In [21]:
# Baixando os dados das estações que serão utilizadas no trabalho
# As estações foram selecionadas a partir do sistema Data Rhama
# Aqui eu baixo os dados e salvo localmente
# >>>>>>>>>>>>> SÓ PRECISA FAZER ISSO UMA VEZ, POR ISSO O CÓDIGO FICA COMENTADO DEPOIS DE RODAR!!!! <<<<<<<<<<<<<

estacao_principal = '56994500'
outras_estacoes = ['01941010', '01941004', '01941006', '56989400', '56989900', '56990000', '56990850', '56990005']

## Telemétricas

In [22]:
# Aplicando a lib HydroBR eu desejo saber se as estações em questão são do tipo convencional ou telemétrica
# O código não exclui o fato, eventual, de uma dada estação ser convencional E telemétrica, como é o caso aqui

lista_estacoes = hbr.get_data.ANA.list_telemetric() # Vendo primeiro se tem telemétrica
lista_estacoes.head()

4733it [00:02, 1776.09it/s]


,Name,Code,Status,SubBasin,City-State,Origem,Responsible,Elevation,Latitude,Longitude
1,RIO PRETO DA EVA,00259004,Ativo,15,RIO PRETO DA EVA-AM,Açudes Semiárido,00001 - ANA - Agência Nacional de Águas,0.00,-2.7003,-59.6997
2,UHE BELO MONTE BR230,00351004,Ativo,18,VITÓRIA DO XINGU-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,33.00,-3.1267,-51.7906
3,UHE BELO MONTE SÍTIO PIMENTAL,00351005,Ativo,18,VITÓRIA DO XINGU-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,110.00,-3.3758,-51.9403
4,UHE BELO MONTE VISTA ALEGRE,00352009,Ativo,18,ALTAMIRA-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,125.00,-3.1186,-52.2525
5,UHE BELO MONTE SÃO FRANCISCO,00352010,Ativo,18,ALTAMIRA-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,124.00,-3.2533,-52.3489


In [23]:
# Averiguando se as estações que tenho em mãos estão presentes neste conjunto de estações telemétricas

print("Estação {e} -> {p}".format(
                                e=estacao_principal,
                                p=(lista_estacoes['Code'] == estacao_principal).any()
                            )
)

# A estação principal em questão tem dados telemétricos

Estação 56994500 -> True


In [24]:
# Verificando as outras estações

for e in outras_estacoes:
    print("Estação {e} -> {p}".format(
                                        e=e,
                                        p=(lista_estacoes['Code'] == e).any()
                                    )
    )

# Estas estações também têm dados telemétricos

Estação 01941010 -> False
Estação 01941004 -> False
Estação 01941006 -> False
Estação 56989400 -> False
Estação 56989900 -> False
Estação 56990000 -> False
Estação 56990850 -> True
Estação 56990005 -> True


In [25]:
# Gerando um arquivo Excel com os dados das estações telemétricas

estacoes_tele = ["56990850", "56990005"]
gerar_dados_tele(
    estacao_principal=estacao_principal,
    outras_estacoes=estacoes_tele,
    nome_arq="baixo_rio_doce",
    dt_inicio='2013-01-01',
    dt_fim='2023-12-31',
    salvar=True
)

## Convencionais

### Cota/Vazão

In [26]:
# Aplicando a lib HydroBR eu desejo saber se as estações em questão são do tipo convencional ou telemétrica
# O código não exclui o fato, eventual, de uma dada estação ser convencional E telemétrica, como é o caso aqui

lista_estacoes = hbr.get_data.ANA.list_flow( # Verificando se tem estações de cota/vazão primeiro
    state='MINAS GERAIS',
    source='ANA'
)

lista_estacoes.head()

2744it [00:01, 2123.65it/s]


,Name,Code,Type,DrainageArea,SubBasin,City,State,Responsible,Latitude,Longitude
1,SÃO ROQUE DE MINAS,40023000,1,None,40,SÃO ROQUE DE MINAS,MINAS GERAIS,IGAM-MG,-20.3344,-46.4697
2,VARGEM BONITA,40025000,1,299,40,VARGEM BONITA,MINAS GERAIS,ANA,-20.3272,-46.3661
3,IGUATAMA,40027000,1,None,40,IGUATAMA,MINAS GERAIS,IGAM-MG,-20.1717,-45.7261
4,FAZENDA DA BARCA,40030000,1,725,40,SÃO ROQUE DE MINAS,MINAS GERAIS,ANA,-20.1000,-46.3167
5,FAZENDA SAMBURÁ,40032000,1,754,40,SÃO ROQUE DE MINAS,MINAS GERAIS,ANA,-20.1508,-46.3033


In [27]:
# Averiguando se as estações que tenho em mãos estão presentes neste conjunto de estações convencionais de cota/vazão

print(
    "Estação {e} -> {p}".format(
        e=estacao_principal,
        p=(lista_estacoes['Code'] == estacao_principal).any()
    )
)

# A estação principal tem dados convencionais de cota/vazão

Estação 56994500 -> False


In [28]:
# Verificando as outras estações

for e in outras_estacoes:
    print(
        "Estação {e} -> {p}".format(
            e=e,
            p=(lista_estacoes['Code'] == e).any()
        )
    )

# Estas estações também têm dados convencionais de cota/vazão

Estação 01941010 -> False
Estação 01941004 -> False
Estação 01941006 -> False
Estação 56989400 -> True
Estação 56989900 -> True
Estação 56990000 -> True
Estação 56990850 -> True
Estação 56990005 -> True


In [29]:
# Gerando um arquivo Excel com os dados das estações convencionais

estacoes_conv = ["56989400", "56989900", "56990000", "56990850", "56990005"]
gerar_dados_conv(
    estacao_principal=estacao_principal,
    outras_estacoes=estacoes_conv,
    nome_arq="baixo_rio_doce",
    dt_inicio='2013-01-01',
    dt_fim='2023-12-31',
    tp_dados=1, # Cota
    nvl_consistencia='2', # dados consistidos
    salvar=True
)

In [30]:
# Gerando um arquivo Excel com os dados das estações convencionais

gerar_dados_conv(
    estacao_principal=estacao_principal,
    outras_estacoes=estacoes_conv,
    nome_arq="baixo_rio_doce",
    dt_inicio='2013-01-01',
    dt_fim='2023-12-31',
    tp_dados=3, # Vazão
    nvl_consistencia='2', # dados consistidos
    salvar=True
)

### Chuva

In [31]:
lista_estacoes = hbr.get_data.ANA.list_prec(
    state='MINAS GERAIS',
    source='ANA'
)

lista_estacoes.head()

2422it [00:01, 2331.05it/s]


,Name,Code,Type,SubBasin,City,State,Responsible,Latitude,Longitude
1,AÇUDE DO ESTREITO,01442019,2,44,ESPINOSA,MINAS GERAIS,DNOCS,-14.8167,-42.8000
2,ESPIGÃO,01442020,2,44,ESPINOSA,MINAS GERAIS,SUDENE,-14.9833,-42.5667
3,ITAMIRIM,01442021,2,44,ESPINOSA,MINAS GERAIS,SUDENE,-14.7667,-42.8833
4,ESPINOSA,01442022,2,44,ESPINOSA,MINAS GERAIS,SUDENE,-14.9333,-42.8167
5,ESPINOSA,01442025,2,44,ESPINOSA,MINAS GERAIS,INMET,-14.9333,-42.8167


In [32]:
print(
    "Estação {e} -> {p}".format(
        e=estacao_principal,
        p=(lista_estacoes['Code'] == estacao_principal).any()
    )
)

# A estação principal NÃO tem dados convencionais de chuva

Estação 56994500 -> False


In [33]:
# Verificando as outras estações

for e in outras_estacoes:
    print(
        "Estação {e} -> {p}".format(
            e=e,
            p=(lista_estacoes['Code'] == e).any()
        )
    )

# Estas estações têm dados convencionais de chuva

Estação 01941010 -> True
Estação 01941004 -> True
Estação 01941006 -> True
Estação 56989400 -> False
Estação 56989900 -> False
Estação 56990000 -> False
Estação 56990850 -> False
Estação 56990005 -> False


In [34]:
# Gerando um arquivo Excel com os dados das estações convencionais

estacoes_conv = ["01941010", "01941004", "01941006"]
gerar_dados_conv(
    estacao_principal=estacao_principal,
    outras_estacoes=estacoes_conv,
    nome_arq="baixo_rio_doce",
    dt_inicio='2013-01-01',
    dt_fim='2023-12-31',
    tp_dados=2, # Chuva
    nvl_consistencia='2', # dados consistidos
    salvar=True
)

## Juntando os dados

In [36]:
arquivos = ['baixo_rio_doce_dados_chuva_conv.xlsx', 'baixo_rio_doce_dados_tele.xlsx', 'baixo_rio_doce_dados_vazao_conv.xlsx']

In [37]:
# Vou fazer a carga primeiro dos dados telemétricos, porque é onde tem mais informação de uma única vez.
# Depois concateno os outros arquivos. Mas a ordem tanto faz aqui, só estipulei assim porque acho melhor

df = pd.read_excel(arquivos[0], sheet_name=0, index_col=0, header=0, parse_dates=['Data'])

for a in range(1, len(arquivos)):
    df_temp = pd.read_excel(arquivos[a], sheet_name=0, index_col=0, header=0, parse_dates=['Data'])
    df = pd.concat([df, df_temp], axis=1)

df

,c_cv_01941010,c_cv_01941004,c_cv_01941006,t_ct_56994500,t_cv_56994500,t_vz_56994500,t_ct_56990850,t_cv_56990850,t_vz_56990850,t_ct_56990005,t_cv_56990005,t_vz_56990005,c_vz_56994500,c_vz_56989400,c_vz_56989900,c_vz_56990000,c_vz_56990850,c_vz_56990005
Data,,,,,,,,,,,,,,,,,,
2013-01-01,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,359.721,15.0942,NaN,45.3207,NaN,NaN
2013-01-02,4.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369.441,14.7624,NaN,38.2480,NaN,NaN
2013-01-03,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,399.210,14.4332,NaN,40.5407,NaN,NaN
2013-01-04,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369.441,13.7821,NaN,41.7113,NaN,NaN
2013-01-05,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350.104,13.7821,NaN,38.2480,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,NaN,NaN,3.6,145.010417,1.8,357.839583,8988.958333,1.6,323.387500,98.958333,0.4,42.700000,324.263,NaN,NaN,NaN,NaN,NaN
2023-12-28,NaN,NaN,0.0,144.031250,0.0,351.442708,8988.500000,0.0,340.245833,98.208333,0.0,41.113043,331.514,NaN,NaN,NaN,NaN,NaN
2023-12-29,NaN,NaN,0.0,152.586957,0.0,394.865217,8982.125000,0.0,446.204167,102.791667,0.0,51.043478,386.241,NaN,NaN,NaN,NaN,NaN


In [38]:
df.columns

Index(['c_cv_01941010', 'c_cv_01941004', 'c_cv_01941006', 't_ct_56994500',
       't_cv_56994500', 't_vz_56994500', 't_ct_56990850', 't_cv_56990850',
       't_vz_56990850', 't_ct_56990005', 't_cv_56990005', 't_vz_56990005',
       'c_vz_56994500', 'c_vz_56989400', 'c_vz_56989900', 'c_vz_56990000',
       'c_vz_56990850', 'c_vz_56990005'],
      dtype='object')

In [39]:
# Vou remover as colunas das cotas

df = df.drop(columns=['t_ct_56994500', 't_ct_56990850', 't_ct_56990005'])
df

,c_cv_01941010,c_cv_01941004,c_cv_01941006,t_cv_56994500,t_vz_56994500,t_cv_56990850,t_vz_56990850,t_cv_56990005,t_vz_56990005,c_vz_56994500,c_vz_56989400,c_vz_56989900,c_vz_56990000,c_vz_56990850,c_vz_56990005
Data,,,,,,,,,,,,,,,
2013-01-01,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,359.721,15.0942,NaN,45.3207,NaN,NaN
2013-01-02,4.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,369.441,14.7624,NaN,38.2480,NaN,NaN
2013-01-03,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,399.210,14.4332,NaN,40.5407,NaN,NaN
2013-01-04,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,369.441,13.7821,NaN,41.7113,NaN,NaN
2013-01-05,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,350.104,13.7821,NaN,38.2480,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,NaN,NaN,3.6,1.8,357.839583,1.6,323.387500,0.4,42.700000,324.263,NaN,NaN,NaN,NaN,NaN
2023-12-28,NaN,NaN,0.0,0.0,351.442708,0.0,340.245833,0.0,41.113043,331.514,NaN,NaN,NaN,NaN,NaN
2023-12-29,NaN,NaN,0.0,0.0,394.865217,0.0,446.204167,0.0,51.043478,386.241,NaN,NaN,NaN,NaN,NaN


In [40]:
df.columns

Index(['c_cv_01941010', 'c_cv_01941004', 'c_cv_01941006', 't_cv_56994500',
       't_vz_56994500', 't_cv_56990850', 't_vz_56990850', 't_cv_56990005',
       't_vz_56990005', 'c_vz_56994500', 'c_vz_56989400', 'c_vz_56989900',
       'c_vz_56990000', 'c_vz_56990850', 'c_vz_56990005'],
      dtype='object')

In [42]:
# Fazendo o merge das colunas de vazão que são correspondentes à mesma estação
# Acontece que existem gaps entre os dados, o que é estranho, porque a estação telemétrica tem dados que a convencional não tem e vice-versa.
# Vou contar qual coluna tem mais dados e depois executar um 'fillna'

colunas_esquerda = ['t_vz_56994500', 't_vz_56990850', 't_vz_56990005']
colunas_direita = ['c_vz_56994500', 'c_vz_56990850', 'c_vz_56990005']

for i, j in zip(colunas_esquerda, colunas_direita):
    print(i, j, df[i].isna().sum(), df[j].isna().sum())

# A coluna que tiver menos buracos será preenchida com os dados da coluna que tem mais dados faltantes

t_vz_56994500 c_vz_56994500 2955 31
t_vz_56990850 c_vz_56990850 1212 4017
t_vz_56990005 c_vz_56990005 1852 4017


In [43]:
# Preenchendo a coluna que tem menos dados faltantes com a outra correspondente
# Fazer isso para cada coluna, contudo, tem colunas que tem dados faltando demais. Neste caso, darei drop nelas inteiramente.
df['c_vz_56994500'].fillna(df['t_vz_56994500'], inplace=True)
df['t_vz_56990850'].fillna(df['c_vz_56990850'], inplace=True)
df['t_vz_56990005'].fillna(df['c_vz_56990005'], inplace=True)

df['c_vz_56994500'].isna().sum(), df['t_vz_56990850'].isna().sum(), df['t_vz_56990005'].isna().sum()

(3, 1212, 1852)

In [44]:
# Remove as colunas que foram usadas para preencher os vazios

df = df.drop(columns=["t_vz_56994500", "c_vz_56990850", "c_vz_56990005"])
df

,c_cv_01941010,c_cv_01941004,c_cv_01941006,t_cv_56994500,t_cv_56990850,t_vz_56990850,t_cv_56990005,t_vz_56990005,c_vz_56994500,c_vz_56989400,c_vz_56989900,c_vz_56990000
Data,,,,,,,,,,,,
2013-01-01,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,359.721,15.0942,NaN,45.3207
2013-01-02,4.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,369.441,14.7624,NaN,38.2480
2013-01-03,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,399.210,14.4332,NaN,40.5407
2013-01-04,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,369.441,13.7821,NaN,41.7113
2013-01-05,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,350.104,13.7821,NaN,38.2480
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,NaN,NaN,3.6,1.8,1.6,323.387500,0.4,42.700000,324.263,NaN,NaN,NaN
2023-12-28,NaN,NaN,0.0,0.0,0.0,340.245833,0.0,41.113043,331.514,NaN,NaN,NaN
2023-12-29,NaN,NaN,0.0,0.0,0.0,446.204167,0.0,51.043478,386.241,NaN,NaN,NaN


In [48]:
# Verificando a quantidade de valores "NaN" em cada coluna

print("Quantidade de NaN por coluna")
print(df.isna().sum())

print("Percentual de NaN por coluna")
for c in np.asarray(df.columns):
    print(c, (df[c].isna().sum()/len(df))*100)

Quantidade de NaN por coluna
c_cv_01941010     153
c_cv_01941004      31
c_cv_01941006       0
t_cv_56994500     973
t_cv_56990850     883
t_vz_56990850    1212
t_cv_56990005    1395
t_vz_56990005    1852
c_vz_56994500       3
c_vz_56989400     366
c_vz_56989900    1520
c_vz_56990000     154
dtype: int64
Percentual de NaN por coluna
c_cv_01941010 3.8088125466766245
c_cv_01941004 0.7717201891959173
c_cv_01941006 0.0
t_cv_56994500 24.22205626089121
t_cv_56990850 21.981578292257904
t_vz_56990850 30.17176997759522
t_cv_56990005 34.727408513816286
t_vz_56990005 46.10405775454319
c_vz_56994500 0.07468259895444362
c_vz_56989400 9.111277072442121
c_vz_56989900 37.83918347025143
c_vz_56990000 3.8337067463281054


In [49]:
# Deixando os dados contínuos, numa base diária.
df = df.resample('D').first()
df

,c_cv_01941010,c_cv_01941004,c_cv_01941006,t_cv_56994500,t_cv_56990850,t_vz_56990850,t_cv_56990005,t_vz_56990005,c_vz_56994500,c_vz_56989400,c_vz_56989900,c_vz_56990000
Data,,,,,,,,,,,,
2013-01-01,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,359.721,15.0942,NaN,45.3207
2013-01-02,4.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,369.441,14.7624,NaN,38.2480
2013-01-03,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,399.210,14.4332,NaN,40.5407
2013-01-04,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,369.441,13.7821,NaN,41.7113
2013-01-05,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,350.104,13.7821,NaN,38.2480
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,NaN,NaN,3.6,1.8,1.6,323.387500,0.4,42.700000,324.263,NaN,NaN,NaN
2023-12-28,NaN,NaN,0.0,0.0,0.0,340.245833,0.0,41.113043,331.514,NaN,NaN,NaN
2023-12-29,NaN,NaN,0.0,0.0,0.0,446.204167,0.0,51.043478,386.241,NaN,NaN,NaN


In [50]:
# Neste momento, tenho o DataFrame com os dados EXATAMENTE da forma que preciso.
# Posso, inclusive, exportar isso para um arquivo de Excel
# É o que farei, pois se precisar retornar aos dados originais, será mais fácil que fazer toda engenharia até aqui

df.to_excel('baixo_rio_doce_final.xlsx')